In [1]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import scipy.fft as fft
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from scipy.stats import levy
from iminuit import Minuit


Initial parameters

In [2]:
s_dim = 750 #Length and height
w_dim = 75 #Depth

N = 10 #Number of particles
radius = 20 #[nm]
s_bounds = (0 + radius, s_dim - radius) #Boundaries
w_bounds =(0 + radius, w_dim - radius) #Boundaries
diff = 1.0 #Diffusion coeff


dev = 50 #std for normal distribution
dev_z = 20
roi_params = [int(s_dim / 10), int(s_dim / 3)] #roi

p_amount = int(N * 0.2) #How many particles to move
step_size = [1,5] #Size of step

n_frames = 80 #Amoun of images to generate
sl = int(w_dim / 2)

In [3]:
conc = (N * (4/3) * np.pi * radius ** 3) / (s_dim ** 2 * w_dim) 
conc

0.007943187351298637

Create initial sample

In [4]:
np.linalg.norm(np.array([0,0,0]) - np.array([5,5,5]))

8.660254037844387

In [5]:
def p_locs(N, s_dim, min_distance):
    '''
    Creates an array of particle locations with:
    N -> amount of particles
    s_dim -> dimensions
    min_distance -> minimum distance between particles
    radius -> radius of the particles
    s_bounds -> spatial bounds for x and y dimensions
    w_bounds -> spatial bounds for z dimension
    dev -> standard deviation for normal distribution
    '''
    
    positions = []
    center = [(s_bounds[0] + s_bounds[1]) / 2, (s_bounds[0] + s_bounds[1]) / 2, 
              (w_bounds[0] + w_bounds[1]) / 2]

    while len(positions) < N:
        # Generate new position with a normal distribution centered at the center
        xy = np.random.normal(loc = center[0], scale = dev, size = (2,))
        zz = np.random.normal(loc = center[2], scale = dev_z, size = (1,))
        
        new_pos = np.hstack((xy, zz))
        
        # Ensure new position is within bounds
        if ((s_bounds[0] + radius <= new_pos[0] <= s_bounds[1] - radius) and 
            (s_bounds[0] + radius <= new_pos[1] <= s_bounds[1] - radius) and 
            (w_bounds[0] + radius <= new_pos[2] <= w_bounds[1] - radius)):
            
            # Check distances from existing particles
            if len(positions) > 0:
                distances = np.linalg.norm(new_pos - np.array(positions), axis=1)
                if np.all(distances >= min_distance):
                    positions.append(new_pos)
            else:
                positions.append(new_pos)

    return np.array(positions)

def sample(locs): 
    '''
    Places the particle's location in a matrix - creating the sample matrix
    '''
    sample = np.zeros((s_dim, s_dim, w_dim))
    for i in range(len(locs)):
        xx, yy, zz = np.meshgrid(range(s_dim), range(s_dim), range(w_dim))
        dist_squared = (xx - locs[:,0][i]) ** 2 + (yy - locs[:,1][i] ) ** 2 + (zz - locs[:,2][i]) ** 2
        sample[dist_squared <= radius ** 2] = 1
    
    return sample

In [6]:
def sample1(locs, radius):
    '''
    Places the particle's location in a matrix
    locs: array of particle locations
    radius: radius of particles
    '''
    sample = np.zeros((s_dim, s_dim, w_dim))
    xx, yy, zz = np.meshgrid(np.arange(s_dim), np.arange(s_dim), np.arange(w_dim), indexing='ij')
    
    for loc in locs:
        dist_squared = (xx - loc[0]) ** 2 + (yy - loc[1]) ** 2 + (zz - loc[2]) ** 2
        sample[dist_squared <= radius ** 2] = 1
    
    return sample

In [7]:
#x,y,z = np.random.normal(loc = 3, scale = 0, size=3)

#(x**2 + y**2 + z**2) ** (1/3)

In [8]:
def brown_motion(ip, step, diff, bounds, width_bounds, p_amount):
    '''
    Function to give particles brownian motion.
    ip: initial position of particles
    step: max and minimum size of steps particles can take
    diff: diffusion factor
    bounds, width_bounds: boundaries
    p_amount: amount of particles being moved
    
    If particles collide / overlap, particle will return to its initial location
    '''
    
    # Set up
    old_positions = np.copy(ip)
    new_positions = np.copy(ip)  
    
    # Moving x amount of particles
    parts = np.random.randint(0, len(ip), p_amount)  # which particles to move

    for i in range(len(parts)):
        step_size = np.random.randint(step[0], step[1], 1)  # Step size of particle
        p_step = np.random.normal(scale = np.sqrt(2 * diff * step_size), size=3)  # Change coordinate
        new_positions[parts[i]] = ip[parts[i]] + p_step
            
        # Boundary conditions - periodic
        for j in range(2):  # Loop over x, y dimensions
            if new_positions[parts[i]][j] + radius > bounds[1]:
                new_positions[parts[i]][j] = bounds[0] + radius
            elif new_positions[parts[i]][j] - radius < bounds[0]:
                new_positions[parts[i]][j] = bounds[1] - radius
        
        # Boundary conditions for depth
        if new_positions[parts[i]][2] + 2 * radius > width_bounds[1]:
            new_positions[parts[i]][2] = width_bounds[1] - radius
        elif new_positions[parts[i]][2] - radius < width_bounds[0]:
            new_positions[parts[i]][2] = width_bounds[0] + radius  
                
    # Collision avoidance in 3D
    for i in range(len(new_positions)):
        for j in range(i + 1, len(new_positions)):
            dist = np.linalg.norm(new_positions[i] - new_positions[j])
            if dist == 0:
                dist = 2 * radius + 1 # when i = j, coords will be equal, this ignores that
            elif dist < 2 * radius + 1:
                # Calculate the direction of collision
                #direction = (new_positions[j] - new_positions[i]) / dist
                # Move the particles back along the collision direction
                #new_positions[i] -= direction * (2 * radius - dist) / 2
                #new_positions[j] += direction * (2 * radius - dist) / 2
                new_positions[i] = old_positions[i]
    
    
        return new_positions, old_positions


In [9]:
def brown_motion_temp(ip, step, diff, bounds, width_bounds, p_amount, temp, radius):
    '''
    Function to give particles brownian motion with a temperature factor.
    ip: initial position of particles
    step: max and minimum size of steps particles can take
    diff: diffusion factor
    bounds, width_bounds: boundaries
    p_amount: amount of particles being moved
    temp: temperature of particles
    radius: radius of particles
    
    If particles collide / overlap, particle will return to its initial location
    '''
    
    # Set up
    old_positions = np.copy(ip)
    new_positions = np.copy(ip)  
    
    # Moving x amount of particles
    parts = np.random.randint(0, len(ip), p_amount)  # which particles to move

    for i in range(len(parts)):
        temp_factor = np.sqrt(abs(np.random.normal(temp, scale = 10)))
        step_size = np.random.randint(int(step[0] + 0.5 * temp_factor),
                                      int(step[1] + 0.5 * temp_factor) , 1)  # Step size of particle
        
        p_step = np.random.normal(scale = np.sqrt(2 * diff * step_size), size=3)  # Change coordinate
        new_positions[parts[i]] = ip[parts[i]] + p_step
            
        #print(p_step)    
            
        # Boundary conditions - periodic
        for j in range(2):  # Loop over x, y dimensions
            if new_positions[parts[i]][j] + radius > bounds[1]:
                new_positions[parts[i]][j] = bounds[0] + radius
            elif new_positions[parts[i]][j] - radius < bounds[0]:
                new_positions[parts[i]][j] = bounds[1] - radius
        
        # Boundary conditions for depth
        if new_positions[parts[i]][2] + 2 * radius > width_bounds[1]:
            new_positions[parts[i]][2] = width_bounds[1] - radius
        elif new_positions[parts[i]][2] - radius < width_bounds[0]:
            new_positions[parts[i]][2] = width_bounds[0] + radius  
                
    # Collision avoidance in 3D
    for i in range(len(new_positions)):
        for j in range(i + 1, len(new_positions)):
            dist = np.linalg.norm(new_positions[i] - new_positions[j])
            if dist == 0:
                dist = 2 * radius + 1 # when i = j, coords will be equal, this ignores that
            elif dist < 2 * radius + 1:
                # Calculate the direction of collision
                #direction = (new_positions[j] - new_positions[i]) / dist
                ## Move the particles back along the collision direction
                #new_positions[i] -= direction * (2 * radius - dist) / 2
                #new_positions[j] += direction * (2 * radius - dist) / 2
                new_positions[i] = old_positions[i]
    
    
        return new_positions, old_positions

    

In [ ]:
locs = p_locs(N, s_dim, (2 * radius))

In [ ]:
b_locs, old_locs = brown_motion(locs, step_size, diff, s_bounds, w_bounds, p_amount)

Illustration of one time step

In [ ]:
fig = plt.figure(figsize = (6,6))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(locs[:,0], locs[:,1], locs[:,2], s = 50, c = 'red', label = 'old position')
ax.scatter(b_locs[:,0], b_locs[:,1], b_locs[:,2], s = 50, c = 'g', label = 'new position')
ax.set_xlim(s_bounds)
ax.set_ylim(s_bounds)
ax.set_zlim(w_bounds)
plt.legend()

In [ ]:
plt.imshow(sample(locs)[:,:,sl])

Create frames of particles moving

In [ ]:
def frame_generator(locs, n_frames):
    '''
    Takes initial locations and moves them around, storing each step in a 'frame'
    and updating initial locations to the new locations.
    locs: initial locations
    n_frames: amount of frames to gather
    '''
    n = 0
    il = np.copy(locs)
    nl = []
    while n < n_frames:
        new_l, old_l = brown_motion(il, step_size, diff, 
                                    s_bounds, w_bounds, p_amount)
        nn = np.vstack(new_l)
        nl.append(nn)
        il = nn
        n += 1
        
    frames = []
    for i in range(len(nl)):
        fr = sample(nl[i])
        frames.append(fr)
        print(i)
    return np.array(frames)

In [ ]:
def frame_generator_temp(locs, n_frames, t_min, t_max):
    '''
    Takes initial locations and moves them around, storing each step in a 'frame'
    and updating initial locations to the new locations.
    locs: initial locations
    n_frames: amount of frames to gather
    '''
    n = 0
    il = np.copy(locs)
    nl = []
    temps = np.linspace(t_min, t_max, n_frames)
    rad = np.linspace(radius + (t_min / 10), radius + (t_max / 10), n_frames)
    while n < n_frames:
        new_l, old_l = brown_motion_temp(il, step_size, diff, 
                                    s_bounds, w_bounds, p_amount,
                                   temps[n], rad[n])
        nn = np.vstack(new_l)
        nl.append(nn)
        il = nn
        n += 1
        
    frames = []
    for i in range(len(nl)):
        fr = sample1(nl[i], rad[i])
        frames.append(fr)
        print(i)
    return np.array(frames)

In [ ]:
frames_t = frame_generator(locs, n_frames)
#frames_t = frame_generator_temp(locs, n_frames, 10, 20) 

In [ ]:
plt.imshow(frames_t[10,:,:,sl])

In [ ]:
plt.imshow(frames_t[0,:,:,sl])

Simulation of movement

In [ ]:
def animation(frames, sl, show = True):
    if show == True:
        num_frames = len(frames)
        fig, ax = plt.subplots()
        im = ax.imshow(frames[0,:,:,sl], cmap='viridis')
        temp_text = ax.text(0.05, 0.95, '', transform = ax.transAxes, 
                              color = 'white', fontsize = 12, ha = 'left', va = 'top')
        #rad_text = ax.text(0.1, 0.95, '', transform = ax.transAxes,
        #                      color = 'white', fontsize = 12, ha = 'left', va = 'top')
        
        def update(frame):
            im.set_array(frames[frame,:,:,sl]) 
            #temp_text.set_text(f'Temperature: {np.round(temps[frame], 2)} K')
            #rad_text.set_text(f'Radius: {np.round(rad[frame], 2)} nm')
            return [im]
    
        ani = FuncAnimation(fig, update, frames=num_frames, interval=150, blit=True)
        return HTML(ani.to_jshtml())
    else:
        return plt.imshow(frames[0])

In [ ]:
animation(frames_t, sl, show = True)

Simulate beam interaction and calculate scattering intensities

In [ ]:
pixel_size = 2# Pixel size
sample_detector_distance = 8e6  # Sample-detector distance 
wavelength = 1 # Wavelength 

In [ ]:
def specks(frames):
    
    '''
    Takes frames as input and calculates scattering intensities for each slice of the 3d sample
    frames: different sample configurations
    '''
    spekls = []
    speqls = []
    for i in range(len(frames)):
        samp = frames[i]
        
        ny, nx, nz = samp.shape  
        qy, qx, qz = np.meshgrid(np.fft.fftfreq(ny, pixel_size), 
                         np.fft.fftfreq(nx, pixel_size),
                         np.fft.fftfreq(nz, pixel_size), indexing='ij')


        q = np.sqrt(qx ** 2 + qy ** 2 + qz ** 2)

        
        sc_intens = np.abs(fft.fftn(samp)) ** 2  
        sc_intens_q = np.fft.fftshift(fft.fftn(samp))  

       
        beam_profile = np.exp(- (q ** 2) / (4 * (1 / wavelength) ** 2))

        
        intens = fft.fftshift(fft.ifftn(fft.fftn(sc_intens) * fft.fftn(beam_profile)))
        intens_q = fft.fftshift(fft.ifftn(fft.fftn(sc_intens_q) * fft.fftn(beam_profile)))

        
        spek = np.abs(intens) ** 2  
        speq = np.abs(intens_q) ** 2  
        
        spekls.append(spek)
        speqls.append(speq)
        
        print(i)
        
    return np.array(spekls), np.array(speqls)

In [ ]:
#spk, spq = specks(frames)
spk_t, spq_t = specks(frames_t)

In [ ]:
plt.figure(figsize = (7,7))
plt.imshow(np.log(spq_t[36,:,:,sl]), cmap = 'viridis')
#plt.xlim(100,150)
#plt.ylim(100,150)

In [ ]:
avg = np.mean(spq_t[:,:,:,sl-2:sl+2], axis = 3)

In [ ]:
plt.imshow(np.log(avg[36]))

Analysis: Azimuthal, TTCF, and g2

In [ ]:
def azimuthal_int(image):
    '''
    Calculates azimuthal integration of SAXS pattern
    image: the frame you want to look at
    '''
    im = image
    center = (s_dim / 2, s_dim / 2)
    d = m.floor(np.sqrt(center[0] ** 2 + center[1] ** 2))
    
    centered = np.meshgrid(np.arange(s_dim) - center[0], np.arange(s_dim) - center[1])
    rad = np.sqrt((centered[0] ** 2 + (centered[1] ** 2)))
                  
    r = np.linspace(1, int(center[0]) + 1, int(center[0]))
                  
    d = lambda r : im [(rad >= r - 0.5) & (rad < r + 0.5)].mean()
                  
    mean_d = np.vectorize(d)(r)
    
    return mean_d

In [ ]:
plt.plot(azimuthal_int(np.log((np.array(spq_t)[10,:,:,sl]))))
plt.xscale('log')

In [ ]:
plt.plot(azimuthal_int(np.log(avg[10])))
plt.xscale('log')

In [ ]:
def roi(ran, speqs, sl, avg = True):
    if avg == True:
        center = [s_dim / 2, s_dim / 2]
        x, y = np.ogrid[:s_dim, :s_dim]
        c_dist = np.sqrt((x - center[0]) ** 2 + (y - center[1]) ** 2)
        mask = (c_dist <= ran[1]) & (c_dist >= ran[0])
        rois = []
        
        for i in range(len(speqs)):
            mat = np.zeros_like(speqs[i])
            mat[mask] = speqs[i][mask]
            rois.append(mat[mask])
        return np.array(rois)
        
    else:
        center = [s_dim / 2, s_dim / 2]
        x, y = np.ogrid[:s_dim, :s_dim]
        c_dist = np.sqrt((x - center[0]) ** 2 + (y - center[1]) ** 2)
        mask = (c_dist <= ran[1]) & (c_dist >= ran[0])
    
        rois = []

        for i in range(len(speqs)):
            mat = np.zeros_like(speqs[i,:,:,sl])
            mat[mask] = speqs[i,:,:,sl][mask]
            rois.append(mat[mask])
        
    return np.array(rois)

In [ ]:
w = roi(roi_params, avg, sl, avg = True)
w_t = roi(roi_params, spq_t, sl, avg = False)

In [ ]:
ttcf_a = np.matmul(w, w.T)
ttcf_t = np.matmul(w_t, w_t.T)

In [ ]:
plt.figure(figsize = (6,6))
plt.imshow(ttcf_t / ttcf_t.max(), origin = 'lower', cmap = 'jet')
plt.colorbar()

In [ ]:
def get_cf(ttcf,skip_last=0):
    '''
    Get g2 function from ttcf
    '''
    return np.array([np.nanmean(np.diagonal(ttcf,offset=i)) for i in range(1,len(ttcf)-skip_last)])

def e_decay(dt,Gm,beta,off):
    '''
    Exponential decay fit function for g2
    '''
    g1 = np.exp(-Gm*dt)
    g2 = beta*g1**2+off
    return g2

g2 = get_cf(ttcf_t / ttcf_t.max())
t_steps = np.linspace(0, len(g2), len(g2))

y_err = np.std(g2)

In [ ]:
def chi2(Gm,beta,off):
    '''
    chi2 function for fitting
    '''
    y_exp = e_decay(t_steps,Gm,beta,off)
    chi2 = np.sum(((g2 - y_exp) / y_err) ** 2)
    return chi2

In [ ]:
fit = Minuit(chi2, beta = g2[0], Gm = 5, off = min(g2))
fit.migrad()

In [ ]:
Gm, beta, off = fit.values

curve_fit = e_decay(t_steps, Gm, beta, off)
chi2_val = np.round(fit.fval, 2)

plt.scatter(t_steps, get_cf(ttcf_t/ttcf_t.max()), s = 10, alpha = 0.5)
plt.plot(curve_fit, color = 'r')
plt.ylabel('g2 - 1')
plt.xlabel(r'$\Delta t \quad [A.U]$')
plt.xscale('log')
plt.text(x = len(curve_fit) - len(curve_fit)/2, y = curve_fit[1],
         s = r'$\chi^{} = {}$'.format(2,chi2_val));

In [ ]:
fit_params = ['Gm: Decay rate','beta: Contrast','off: Offset']
for i in range(len(fit.values)):
    print(fit_params[i],np.round(fit.values[i],4))